In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd 

df = pd.read_csv('/kaggle/input/amazon-books-reviews/Books_rating.csv')

In [3]:
df.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [4]:
new_df = df.drop(columns=['Title', 'Price', 'profileName', 'review/helpfulness', 'review/time', 'review/summary', 'review/text'], axis=1)

In [5]:
new_df = new_df.dropna()

In [6]:
new_df.head()

,Id,User_id,review/score
0,1882931173,AVCGYZL8FQQTD,4.0
1,0826414346,A30TK6U7DNS82R,5.0
2,0826414346,A3UH4UZ4RSVO82,5.0
3,0826414346,A2MVUWT453QH61,4.0
4,0826414346,A22X4XUPKF66MR,4.0


In [7]:
#No of Users
N_unique = new_df.User_id.unique()
#No of books 
M_unique = new_df.Id.unique()

print(N_unique.size)
print(M_unique.size)

1008972
216023


In [65]:
from collections import Counter

user_ids_count = Counter(new_df.User_id)
book_ids_count = Counter(new_df.Id)

# number of users and books we would like to keep
n = 3000
m = 1000

#Too much lets do a small size database processing 
user_ids = [u for u, c in user_ids_count.most_common(n)]
book_ids = [m for m, c in book_ids_count.most_common(m)]

# make a copy, otherwise ids won't be overwritten
df_small = new_df[new_df.User_id.isin(user_ids) & new_df.Id.isin(book_ids)].copy()

df_small.describe()

,review/score
count,92644.00000
mean,4.37928
std,0.95031
min,1.00000
25%,4.00000
50%,5.00000
75%,5.00000
max,5.00000


In [66]:
df_small.head()

,Id,User_id,review/score
1222,B0007H4QBK,AF3X7J0XC391L,5.0
1224,B0007H4QBK,A3FHSO1SKHU378,5.0
1244,B0007H4QBK,A16QJ649N8PRV,5.0
1249,B0007H4QBK,A321W4SSC0F6AP,5.0
1260,B0007H4QBK,A3JPFWKS83R49V,4.0


In [68]:
#No of Users
N_small = df_small.User_id.unique()
#No of books 
M_small = df_small.Id.unique()


In [69]:
new_book_id_map = {}
j = 0
for old in M_small:
  new_book_id_map[old] = j
  j += 1
print("j:", j)

print("Setting new ids")
df_small.loc[:, 'Id'] = df_small.apply(lambda row: new_book_id_map[row.Id], axis=1)


j: 997
Setting new ids


In [70]:
# need to remake user ids and book ids since they are no longer sequential
new_user_id_map = {}
i = 0
for old in N_small:
  new_user_id_map[old] = i
  i += 1
print("i:", i)

def convert_to_int(string):
  result =  new_user_id_map.get(string)
  return result

df_small.loc[:, 'User_id'] = df_small.apply(lambda row: convert_to_int(row.User_id), axis=1)

i: 2856


In [71]:
df_small.head()

,Id,User_id,review/score
1222,0,0,5.0
1224,0,1,5.0
1244,0,2,5.0
1249,0,3,5.0
1260,0,4,4.0


In [72]:
from sklearn.utils import shuffle

N = df_small.User_id.max() + 1 # number of users
M = df_small.Id.max() + 1 # number of movies
print(N)
print(M)

print(len(df_small))
# split into train and test
df_small = shuffle(df_small)
cutoff = int(0.8*len(df_small))
print(cutoff)
df_train = df_small.iloc[:cutoff]
df_test = df_small.iloc[cutoff:]

2856
997
92644
74115


In [59]:
# No of users we would want to consider 
K = 25 # number of neighbors we'd like to consider
limit = 5 # number of common books users must have in common in order to consider

N_tr = df_train.User_id.max() + 1
print(N_tr)

4826


In [60]:
df_train.head() 
print(df_train.User_id.unique().size)


4794


In [61]:
grouped_by_user = df_train.groupby("User_id")


In [62]:

averages = grouped_by_user["review/score"].agg('mean')

def calculate(dx):
    return dx["review/score"] - averages[dx["User_id"]]
    
df_train["deviations"] = df_train.apply(calculate, axis=1)
    

/tmp/ipykernel_32/251866279.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["deviations"] = df_train.apply(calculate, axis=1)


In [63]:
print(df_train.head())
from sortedcontainers import SortedList
grouped_by_user = df_train.groupby("User_id")
print(len(grouped_by_user))

           Id User_id  review/score  deviations
1311004   885     474           5.0    0.212963
1345929   906     663           4.0   -0.325581
2460449  1626    2379           4.0   -0.309091
2802608  1865     105           5.0    0.218927
2269791  1504    2304           4.0   -0.886792
4794


In [64]:
#find users with more than limit books in common to consider 
#store their correlation and user id with respect to user
def dev(df):
    return df["deviations_x"] * df["deviations_y"]
    
neighbours = {}
for x in grouped_by_user:
    dx = x[1]
    sl = SortedList()
    sigmai = np.sqrt(dx["deviations"].dot(dx["deviations"]))
    for y in grouped_by_user:
        if (x[0]!=y[0]):
            dy = y[1]
            merged = pd.merge(dx, dy, on='Id', how='inner');
            if merged["Id"].count()>limit:
                #denominator
                sigmaj = np.sqrt(dy["deviations"].dot(dy["deviations"]))
                denominator = sigmaj * sigmai 
                #numerator
                numerator = merged.apply(dev, axis=1).sum()
                #correlation coefficient
                if denominator != 0:
                    w_ij = numerator / denominator
                else:
                    w_ij = 0
                    
                sl.add((-w_ij, y[0]))
                if len(sl) > K:
                    del sl[-1] 
      
    # store the neighbors
    print(x[0])
    neighbours[x[0]] = sl
    # print out useful things
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77


KeyboardInterrupt: 

In [47]:
def predict(i, m):
  # calculate the weighted sum of deviations
  numerator = 0
  denominator = 0
  if i not in neighbours.keys():
    return 2.5
  for neg_w, j in neighbours[i]:
    # remember, the weight is stored as its negative
    # so the negative of the negative weight is the positive weight
    try:
      numerator += -neg_w * grouped_by_user[i]["Id"==j]["deviations"]
      denominator += abs(neg_w)
    except KeyError:
      # neighbor may not have rated the same movie
      # don't want to do dictionary lookup twice
      # so just throw exception
      pass

  if denominator == 0:
    prediction = averages[i]
  else:
    prediction = numerator / denominator + averages[i]
  prediction = min(5, prediction)
  prediction = max(0.5, prediction) # min rating is 0.5
  return prediction


In [48]:
train_predictions = []
train_targets = []
for index,values in df_train.iterrows():
  # calculate the prediction for this movie
  i = values["User_id"]
  m = values["Id"]
  target = values["review/score"]
  prediction = predict(i, m)

  # save the prediction and target
  train_predictions.append(prediction)
  train_targets.append(target)



In [50]:
test_predictions = []
test_targets = []
# same thing for test set
for index, values in df_test.iterrows():
  # calculate the prediction for this movie
  i = values["User_id"]
  m = values["Id"]
  target = values["review/score"]
  prediction = predict(i, m)

  # save the prediction and target
  test_predictions.append(prediction)
  test_targets.append(target)

In [51]:
# calculate accuracy
def mse(p, t):
  p = np.array(p)
  t = np.array(t)
  return np.mean((p - t)**2)

print('train mse:', mse(train_predictions, train_targets))
print('test mse:', mse(test_predictions, test_targets))

train mse: 0.5285414721738041
test mse: 0.5539022615385794
